<a href="https://colab.research.google.com/github/jbardsle/URSP688YFinal/blob/main/Github_version_of_FinalProjectPaperVersion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import geopandas as gpd
import os
import numpy as np
import matplotlib.pyplot as plt
import shapely
import os


In [ ]:
import sys
sys.path.append('URSP688Final/src')

In [ ]:
from modules.clean_pcts import clean_pcts
from modules.dist_decay import dist_decay

In [ ]:
OppSites = gpd.read_file('data/OpportunitySites2/OpportunitySites2.shp')

In [ ]:
# This begins the process to clean the original large data files. For loading speed,
# the smaller versions are loaded below, and this section is commented out.
# OppSites is small enough that it is loaded below.
# PgParcels = gpd.read_file('data/Property_Info_Py (1)/Property_Info_Py.shp')

In [ ]:
# Demogs = gpd.read_file('data/PG_Blocks/PG_Blocks.shp')

In [ ]:
#Select the opportunity sites that have a 3 mile radius completely withing Prince George's County
#and a small sliver of DC
OppSites = OppSites[OppSites['DissolveID'].isin([11,31,188,])]

In [ ]:
#Convert all geodataframes to UTM-18 for distance measurement
# PgParcels = PgParcels.to_crs(epsg = 32618)
OppSites = OppSites.to_crs(epsg = 32618)
# Demogs = Demogs.to_crs(epsg = 32618)

In [ ]:
#Create a buffer around each site of approximately three miles (4828 meters) to represent the market area
OppSitesBuffer = OppSites['geometry'].buffer(4828)

In [ ]:
#Copy the Opportunity Sites layer so that it can be replaced with the buffer geometry
OppSitesWithBuffer = OppSites.copy()

In [ ]:
#Replace original geometry with buffer geometry
OppSitesWithBuffer['geometry'] = OppSitesBuffer

In [ ]:
#plot the buffers with the opportunity sites
ax = OppSitesBuffer.plot()
OppSites.plot(ax=ax, color = 'red')

In [ ]:
# #Create a single shape to represent the maximum area of interest (3 miles from the three sites)
# MaxArea = OppSitesBuffer.unary_union
# #Convert this polygon object to a geodataframe
# MaxArea_series = gpd.GeoSeries(MaxArea)
# MaxAreaGDF = gpd.GeoDataFrame(geometry = MaxArea_series, crs = 32618)

In [ ]:
# PgParcelsSmall = gpd.sjoin(PgParcels, MaxAreaGDF,how="inner", op='intersects')

In [ ]:
# DemogsSmall = gpd.sjoin(Demogs, MaxAreaGDF,how="inner", op='intersects')

In [ ]:
#Load PgParcelsSmall from file (rather than performing the above process)
PgParcelsSmall = gpd.read_file('data/PgParcelsSmall3Miles/PgParcelsSmall3Mile.shp')

In [ ]:
#Load DemogsSmall from file (rather than performing the above process)
DemogsSmall = gpd.read_file('data/DemogsSmall3Miles/DemogsSmall3Mile.shp')

In [ ]:
#Remove all columns except the one containing the Geoid, total pop, hispanic pop, and hispanic percent
DemogsSmall = DemogsSmall[['GEOID','P0010001','P0020002','PCT_P00200','geometry']]

In [ ]:
#Rename columns for ease of use
DemogsSmall.rename(columns={'P0010001': 'Total_Pop', 'P0020002': 'Hisp_Pop', 'PCT_P00200': 'Hisp_Pct'}, inplace=True)

In [ ]:
#Get rid of index_right column in PgParcelsSmall which causes an error in a future join
PgParcelsSmall.drop('index_righ', axis=1, inplace=True)

In [ ]:
#JoinPgParcelsSmall with demographic data
ParcelDemogs = gpd.sjoin(PgParcelsSmall,DemogsSmall,how="inner", op = "intersects")

In [ ]:
# Group by the parcel account number so that there is one record per parcel
ParcelDemogsCln = ParcelDemogs[['ACCT_PRIMA','DUS','GEOID','Total_Pop','Hisp_Pop','Hisp_Pct','geometry']].groupby('ACCT_PRIMA').first()

In [ ]:
#Running the CleanPcts function
clean_pcts(ParcelDemogsCln)

In [ ]:
#Running the dist_decay function with quadratic decline
dist_decay(OppSites,ParcelDemogsCln,2)

In [ ]:
#creating a percent column
OppSites['PctHisp'] = (OppSites['HispCustomers'] / OppSites['Customers'])

In [ ]:
#checking the results
OppSites.head()

In [ ]:
# Exporting the results to csv
# OppSites.to_csv('/content/drive/MyDrive/Colab Notebooks/FinalProject/Results/May10Exponential2.csv')